# FPI Model

### This model compares ESPN FPI (Football Power Index) to Bovada odds and suggests bets

### Organization
1. Get up to date FPI from ESPN- DONE
2. Get up to date odds from Bovada - difficult, use espn chalk instead
3. Money line implied odds comparison
4. Point spread implied odds comparison

_______________________________________________________________________________________________________________________________

In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [3]:
# up to date FPI function
# returns pandas dataframe with current FPI table

def getFPI():
    url = 'https://www.espn.com/nfl/fpi'
    r = requests.get(url)
    html = r.text
    
    # ESPN splits the FPI table into two sides
    
    # put the left table (team names) into a pandas dataframe    
    soup = BeautifulSoup(html)
    table1 = soup.find('table', {"class": "Table Table--align-right Table--fixed Table--fixed-left"})
    rows = table1.find_all('tr')
    teams_data = []
    for row in rows[2:]:
        cols = row.find_all('td')
        cols = [element.text.strip() for element in cols]
        teams_data.append([element for element in cols if element])   

    teams_df = pd.DataFrame(teams_data)
    
    # put the right side table (FPI and other stats) into a pandas dataframe
    table2 = soup.find('table', {"class": "Table Table--align-right"})
    rows = table2.find_all('tr')
    stats_data = []
    for row in rows[2:]:
        cols = row.find_all('td')
        cols = [element.text.strip() for element in cols]
        stats_data.append([element for element in cols if element])   

    stats_df = pd.DataFrame(stats_data)
    
    # combine into one dataframe and update headings
    df = pd.merge(teams_df, stats_df, left_index=True, right_index=True)
    headers = {'0_x': 'TEAM', 
               '0_y': 'W-L', 
               1 : 'FPI', 
               2: 'RK', 
               3: 'TRND', 
               4: 'OFF', 
               5: 'DEF', 
               6: 'ST', 
               7: 'SOS', 
               8: 'REM_SOS', 
               9: 'AVG_WP'}
    df = df.rename(index=str,columns=headers)
    return df

In [4]:
def getESPNlines():
    url = 'https://www.espn.com/nfl/lines'
    dfs = pd.read_html(url)
    
    return dfs

In [5]:
dfs_in = getESPNlines()

In [6]:
line_dfs = dfs_in[:]
for i, df in enumerate(dfs_in):

    line_dfs[i] = df

## How to calculate implied odds (%) based on American Money Line
 
### Negative American Odds (favorite)
implied probability = negative odds / (negative odds + 100) * 100

### Positive American Odds (underdog)
implied probability = 100 / (positive odds + 100) * 100

In [7]:
# add implied odds column to dfs and compare
lines_list = line_dfs[:]
for i, df in enumerate(line_dfs):
    # sort: underdogs, favorites    
    df = df.sort_values('LINE', ascending=False)
    
    # add total and points for each team row
    total, spread = df.iloc[0]['LINE'], df.iloc[1]['LINE']
    df['TOTAL'] = [total, total]
    df['SPREAD'] = [-spread, spread]
    
    # drop unnecessary columns
    df = df.drop(['REC (ATS)', 'OPEN', 'LINE'], axis=1)
  
    # sort: underdog, favorite
    df = df.sort_values(['ML'], ascending=False)
    
    # calculate implied odds from ML
    for index, row in df.iterrows():
        if row['ML'] > 0:
            underdog = (100*100/(row['ML']+100))
        else:
            favorite = 100*abs(row['ML'])/(abs(row['ML'])+100)
    df['Implied_Odds'] = [underdog,favorite]
    
    # calculate ML edge (FPI - Implied Odds)
    df['FPI'] = df['FPI'].str.strip(' %')
    df['FPI'] = df['FPI'].astype(float)
    
    df['ML_Edge'] = df['FPI'] - df['Implied_Odds']
    df['U/F?'] = ['underdog', 'favorite']
    
    df.columns.values[0] = 'TEAM'
    
    lines_list[i] = df

TypeError: bad operand type for unary -: 'str'

In [8]:
# convert list of games to dataframe, organize columns, index by team
lines = pd.concat(lines_list)
lines = lines[['TEAM', 'ML', 'SPREAD', 'TOTAL', 'FPI', 'Implied_Odds', 'ML_Edge', 'U/F?']]
team_lines = lines.set_index('TEAM')

points_FPI = getFPI()
points_FPI = points_FPI.set_index('TEAM')
points_FPI['FPI'] = points_FPI['FPI'].astype(float)
points_FPI

,W-L,FPI,RK,TRND,OFF,DEF,ST,SOS,REM_SOS,AVG_WP
TEAM,,,,,,,,,,
Kansas City Chiefs,4-0,9.8,1,--,9.0,1.0,-0.2,15,28,7
Baltimore Ravens,3-1,7.0,2,--,3.9,2.8,0.4,13,30,5
San Francisco 49ers,2-2,5.5,3,--,2.4,2.8,0.3,31,1,6
Seattle Seahawks,4-0,4.9,4,--,5.3,-0.4,0.0,17,17,4
Green Bay Packers,4-0,4.9,5,--,6.7,-1.5,-0.4,16,23,3
Tampa Bay Buccaneers,3-2,4.5,6,1,0.8,4.2,-0.5,23,11,11
New Orleans Saints,2-2,4.2,7,1,5.0,-1.0,0.2,9,15,12
Los Angeles Rams,3-1,4.0,8,2,3.2,0.8,-0.0,22,6,9
Buffalo Bills,4-0,3.4,9,1,2.2,1.4,-0.2,25,10,1


# NOTE

ESPN SOS (strength of schedule) and AVG_WP (average in-game win probability) appear to be inversely related (intuitive), but discrepencies could lead to betting edge. Room for more analysis

### Money Line Bets Sorted by Best Edge
only displays ML bets with a positive edge

In [9]:
# take positive edge, round values, and organize
pos_edge = team_lines[team_lines.ML_Edge > 0]
pos_edge = pos_edge[['ML', 'FPI', 'Implied_Odds', 'ML_Edge', 'U/F?']]
pos_edge = pos_edge.sort_values('ML_Edge', ascending=False)
pos_edge.round(decimals=3)

,ML,FPI,Implied_Odds,ML_Edge,U/F?
TEAM,,,,,
Cleveland Browns,-110,50.3,22.727,27.573,underdog
New York Jets,280,42.9,26.316,16.584,underdog
Washington,280,33,26.316,6.684,underdog
Minnesota Vikings,260,34,27.778,6.222,underdog
Philadelphia Eagles,270,32.3,27.027,5.273,underdog
Cincinnati Bengals,600,18.3,14.286,4.014,underdog
Atlanta Falcons,-120,55.3,54.545,0.755,favorite
Las Vegas Raiders,475,17.8,17.391,0.409,underdog
Jacksonville Jaguars,200,33.7,33.333,0.367,underdog


### Point Spreads
ESPN FPI is defined as the expected point margin vs an average opponent
Assume an average opponent has an FPI of zero

FPI favorite expected point spread = Favorite FPI - Underdog FPI
FPI underdog expected point spread = -FPI favorite expected point spread

In [10]:
# loop over each matchup and calculate
spread_list = lines_list.copy()
for i, df in enumerate(lines_list):
    
    # drop unnecessary columns
    df = df.drop(['ML', 'FPI', 'TOTAL', 'Implied_Odds', 'ML_Edge'], axis=1)

    # index to teams
    df = df.set_index('TEAM')

    # add FPI, calculate FPI spread, calculate points edge
    df = pd.concat([df, points_FPI['FPI']], axis=1, join='inner')
    fpi_spread = df.iloc[0]['FPI'] - df.iloc[1]['FPI']
    df['FPI_spread'] = [-fpi_spread, fpi_spread]
    df['Points_Edge'] = df['SPREAD'] - df['FPI_spread']
     
    spread_list[i] = df

KeyError: "['TOTAL' 'Implied_Odds' 'ML_Edge'] not found in axis"

In [11]:
# convert list to a dataframe, organize, take positive edge
spreads = pd.concat(spread_list)
spreads = spreads[['SPREAD', 'FPI', 'FPI_spread', 'Points_Edge', 'U/F?']]

points_edge = spreads[spreads.Points_Edge > 0]
points_edge = points_edge.sort_values('Points_Edge', ascending=False)
points_edge

,SPREAD,FPI,FPI_spread,Points_Edge,U/F?
Philadelphia Eagles,7.0,-2.2,3.6,3.4,underdog
Minnesota Vikings,7.0,1,3.9,3.1,underdog
New York Jets,7.5,-6.4,5.1,2.4,underdog
Jacksonville Jaguars,5.5,-6.7,3.8,1.7,underdog
Las Vegas Raiders,12.0,-0.5,10.3,1.7,underdog
Indianapolis Colts,-0.0,1.3,-1.1,1.1,favorite
Los Angeles Rams,-7.0,4,-8.1,1.1,favorite
Cincinnati Bengals,12.0,-4.1,11.1,0.9,underdog
Dallas Cowboys,-8.5,1.4,-9.0,0.5,favorite


__________________________________________________________

# Final Results

## Money Line Edge

In [12]:
pos_edge

,ML,FPI,Implied_Odds,ML_Edge,U/F?
TEAM,,,,,
Cleveland Browns,-110,50.3,22.727273,27.572727,underdog
New York Jets,280,42.9,26.315789,16.584211,underdog
Washington,280,33,26.315789,6.684211,underdog
Minnesota Vikings,260,34,27.777778,6.222222,underdog
Philadelphia Eagles,270,32.3,27.027027,5.272973,underdog
Cincinnati Bengals,600,18.3,14.285714,4.014286,underdog
Atlanta Falcons,-120,55.3,54.545455,0.754545,favorite
Las Vegas Raiders,475,17.8,17.391304,0.408696,underdog
Jacksonville Jaguars,200,33.7,33.333333,0.366667,underdog


## Points Edge

In [13]:
points_edge

,SPREAD,FPI,FPI_spread,Points_Edge,U/F?
Philadelphia Eagles,7.0,-2.2,3.6,3.4,underdog
Minnesota Vikings,7.0,1,3.9,3.1,underdog
New York Jets,7.5,-6.4,5.1,2.4,underdog
Jacksonville Jaguars,5.5,-6.7,3.8,1.7,underdog
Las Vegas Raiders,12.0,-0.5,10.3,1.7,underdog
Indianapolis Colts,-0.0,1.3,-1.1,1.1,favorite
Los Angeles Rams,-7.0,4,-8.1,1.1,favorite
Cincinnati Bengals,12.0,-4.1,11.1,0.9,underdog
Dallas Cowboys,-8.5,1.4,-9.0,0.5,favorite


___________________________________________________________

# Stuff to Add

* add opponent, home/away, record, weather to dataframes
* add strength of schedule to date anaylsis